In [1]:
 !pip install transformers datasets scikit-learn torch

In [2]:
# 2. Import libraries
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, f1_score



D:\anaconda\envs\DNNs\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
# 3. Load dataset
dataset = load_dataset("ag_news")

In [4]:

# 4. Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
# 5. Load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments

In [7]:

# 6. Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
)

In [8]:
!pip install --upgrade transformers


In [9]:
# 7. Define metrics
def compute_metrics(pred):
    predictions = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

In [10]:
# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(100)),
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(50)),
    compute_metrics=compute_metrics
)

In [11]:
# 9. Train model
trainer.train()

Step,Training Loss


TrainOutput(global_step=14, training_loss=1.1612052917480469, metrics={'train_runtime': 1329.858, 'train_samples_per_second': 0.15, 'train_steps_per_second': 0.011, 'total_flos': 52623156019200.0, 'train_loss': 1.1612052917480469, 'epoch': 2.0})

In [12]:

# 10. Evaluate
eval_results = trainer.evaluate()
print("Evaluation:", eval_results)

Evaluation: {'eval_loss': 1.012974739074707, 'eval_accuracy': 0.72, 'eval_f1': 0.6480308880308882, 'eval_runtime': 40.9344, 'eval_samples_per_second': 1.221, 'eval_steps_per_second': 0.024, 'epoch': 2.0}
